In [1]:

import os
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor

# Define paths to input directories
path_to_data_t1 = "/Intern Programming/data/UMCGOPC_MRI_nii_zscoreinHNmask"
path_to_data_t1gt0tt = "/Intern Programming/data/output/t1t2gtv_3mm"
clc_data_path = '/Intern Programming/data/OPCdigits_333_split_t1good.csv'

# Load clinical data
clc_data = pd.read_csv(clc_data_path).drop(columns=['Unnamed: 0'])

# Get list of patient IDs and corresponding files
files = clc_data['PatientID']

# Define lists of CT images and GTV masks
mri_images = [os.path.join(path_to_data_t1, f"{x}_t1.nii.gz") for x in files]
gtv_masks = [os.path.join(path_to_data_t1gt0tt, f"{x}_gtvpttot1.nii.gz") for x in files]

# Create a PyRadiomics feature extractor object
extractor = featureextractor.RadiomicsFeatureExtractor()

# Specify the features you want to extract
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('shape')
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glrlm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('ngtdm')

# Specify the discretization of the image intensities
extractor.settings["binWidth"] = 0.1  # Intensity discretization bin width
extractor.settings["resampledPixelSpacing"] = (0.5, 0.5, 1.0)  # Resampled voxel spacing

# Specify the resampling of the image and mask
extractor.settings["interpolator"] = "sitkLinear"
extractor.settings["resamplingInterpolator"] = "sitkNearestNeighbor"
extractor.settings["correctMask"] = True  # Correct mask if it extends outside the image bounds

# Initialize an empty list to store DataFrames for each patient
feature_dfs = []

# Iterate over patients and extract features
for patient_id, ct_image_path, gtv_mask_path in zip(files, mri_images, gtv_masks):
    try:
        # Load CT image and GTV mask
        ct_image = sitk.ReadImage(ct_image_path)
        gtv_mask = sitk.ReadImage(gtv_mask_path)
        
        # Extract radiomics features using the feature extractor
        features = extractor.execute(ct_image, gtv_mask)
        
        # Add patient ID to features dictionary
        features['PatientID'] = patient_id
        
        # Convert features OrderedDict to DataFrame
        features_df = pd.DataFrame.from_dict(features, orient='index').T
        
        # Append features DataFrame to list of DataFrames
        feature_dfs.append(features_df)
        
        print(f"Successfully extracted features for patient {patient_id}")
    except Exception as e:
        print(f"Error extracting features for patient {patient_id}: {e}")

# Check if any features were successfully extracted
if feature_dfs:
    # Concatenate all feature DataFrames into a single DataFrame
    all_features = pd.concat(feature_dfs, ignore_index=True)
    
    # Print the DataFrame containing all extracted features
    print(all_features)
else:
    print("No features were extracted successfully.")



Error extracting features for patient UMCG-0573601: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\1\sitk-build\ITK\Modules\IO\NIFTI\src\itkNiftiImageIO.cxx:589:
ITK ERROR: NiftiImageIO(000002448407C6F0): nifti_image_load failed for file: /Intern Programming/data/output/t1t2gtv_3mm\UMCG-0573601_gtvpttot1.nii.gz
Error extracting features for patient UMCG-6395952: Bounding box of ROI is larger than image space:
	ROI bounds (x, y, z image coordinate space) ((305.5, 320.5, 0.5), (464.5, 419.5, 74.5))
	Image Size (256, 256, 41)
Error extracting features for patient UMCG-5908531: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\1\sitk-build\ITK\Modules\IO\NIFTI\src\itkNiftiImageIO.cxx:589:
ITK ERROR: NiftiImageIO(00000244843CE4E0): nifti_image_load failed for file: /Intern Programming/data/output/t1t2gtv_3mm\UMCG-5908531_gtvpttot1.nii.gz
Error extracting features for patient UMCG-6754277: Bounding box of ROI is larger than image space:
	ROI bounds (x, y, z image coord

KeyboardInterrupt: 

In [1]:
import os
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor

# Define paths to input directories
path_to_data_t1 = "/Intern Programming/data/UMCGOPC_MRI_nii_zscoreinHNmask"
path_to_data_t1gt0tt = "/Intern Programming/data/output/t1t2gtv_3mm"
clc_data_path = '/Intern Programming/data/OPCdigits_333_split_t1good.csv'

# Load clinical data
clc_data = pd.read_csv(clc_data_path).drop(columns=['Unnamed: 0'])

# Get list of patient IDs and corresponding files
files = clc_data['PatientID']

# Define lists of MRI images and GTV masks
mri_images = [os.path.join(path_to_data_t1, f"{x}_t1r.nii.gz") for x in files]
gtv_masks = [os.path.join(path_to_data_t1gt0tt, f"{x}_gtvpttot1r.nii.gz") for x in files]

# Create a PyRadiomics feature extractor object
extractor = featureextractor.RadiomicsFeatureExtractor()

# Enable all feature classes
extractor.enableAllFeatures()

# Specify the resampling of the image and mask
extractor.settings["interpolator"] = "sitkLinear"
extractor.settings["resamplingInterpolator"] = "sitkNearestNeighbor"
extractor.settings["correctMask"] = True  # Correct mask if it extends outside the image bounds

# Initialize an empty DataFrame to store features
all_features = pd.DataFrame()

# Iterate over patients and extract features
for patient_id, ct_image_path, gtv_mask_path in zip(files, mri_images, gtv_masks):
    try:
        # Load MRI image and GTV mask
        ct_image = sitk.ReadImage(ct_image_path)
        gtv_mask = sitk.ReadImage(gtv_mask_path)

        # Extract radiomics features using the feature extractor
        features = extractor.execute(ct_image, gtv_mask)

        # Add patient ID to features dictionary
        features['PatientID'] = patient_id

        # Convert features OrderedDict to DataFrame
        features_df = pd.DataFrame.from_dict(features, orient='index').T

        # Append features DataFrame to DataFrame containing all features
        all_features = pd.concat([all_features, features_df], ignore_index=True)

        print(f"Successfully extracted features for patient {patient_id}")
    except Exception as e:
        print(f"Error extracting features for patient {patient_id}: {e}")

# Save the DataFrame containing all extracted features to a CSV file
if not all_features.empty:
    output_csv_path = "/Intern Programming/data/radiomics_features.csv"
    all_features.to_csv(output_csv_path, index=False)
    print(f"Successfully saved features to {output_csv_path}")
else:
    print("No features were extracted successfully.")


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-0573601


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-6395952


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


...

Successfully extracted features for patient UMCG-0412861


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-5325319


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-2291679
Successfully saved features to /Intern Programming/data/radiomics_features.csv


In [2]:
df = pd.DataFrame(all_features)
df.to_csv(r'/Intern Programming/data/output/' + 'umcg_radiomicsfeature_t1_GTVPTtot_3mm.csv' )